In [1]:
from langchain_groq import ChatGroq


In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-39013?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for Senior Software Engineer

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySenior Software EngineerBeaverton, OregonBecome a Part of the NIKE, Inc. TeamNIKE, Inc. does

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [5]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Senior Software Engineer',
 'experience': 'Senior-level',
 'skills': ['Python (preferred) or similar object-oriented programming languages',
  'JavaScript front end-web frameworks (e.g. Angular, React)',
  'Data structures and algorithms for efficiently and effectively managing large data sets',
  'Serverless data warehouses (e.g. Athena) and NoSQL databases (e.g. DynamoDB)',
  'GraphQL and RESTful APIs',
  'Cloud infrastructure development using public cloud providers (e.g., AWS, Azure, GCP)',
  'Infrastructure-as-code technologies (e.g., Terraform, CloudFormation, CircleCI)',
  'Continuous integration & deployment (CI/CD), source control (e.g. GitHub), and build automation (e.g. Jenkins, GitHub Actions)',
  'Agile principles and Sprint Planning, including planning/backlog tools (e.g., JIRA)'],
 'description': 'We’re looking for a senior-level Software Engineer with the skills to deliver and maintain product features for our internal cloud reporting solution. This valued tea

In [6]:
type(json_res)

dict

In [7]:
import pandas as pd

df = pd.read_csv("portfolio.csv")
df

,Techstack,Links
0,"Flutter, Firebase, Dart",https://play.google.com/store/apps/details?id=...
1,"Flutter, Firebase, Dart",https://github.com/qusaii21/Ganga-Prabodhanam-App
2,"Machine Learning, Python, TensorFlow",https://github.com/qusaii21/Ganga-Prabodhanam-App
3,"Machine Learning, GANs, Python",https://github.com/qusaii21/Synthetic-Data-Gen...
4,"React Native, MongoDB, Express.js, Node.js, Ja...",https://github.com/qusaii21/Quick-Sewa-On-Dema...
5,"Machine Learning, Python",https://github.com/qusaii21/RouteMate-Optimize...
6,"Flutter, Dart, Data Structures",https://github.com/qusaii21/minimum-cost-of-ti...
7,"Machine Learning, Python, Streamlit, firebase",https://github.com/qusaii21/ANPR-System-for-Ap...
8,"Flutter, Dart, Flame",https://github.com/qusaii21/Tilt-n-Score-game


In [8]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [10]:
links = collection.query(query_texts=["Experience in flutter","Experience in python"], n_results=2).get('metadatas', [])
links


[[{'links': 'https://play.google.com/store/apps/details?id=com.ieeeofficialapp.ieee_official_app'},
  {'links': 'https://github.com/qusaii21/Ganga-Prabodhanam-App'}],
 [{'links': 'https://github.com/qusaii21/RouteMate-Optimized-Employee-Transportation-Routing'},
  {'links': 'https://github.com/qusaii21/Ganga-Prabodhanam-App'}]]

In [11]:
job = json_res
job['skills']

['Python (preferred) or similar object-oriented programming languages',
 'JavaScript front end-web frameworks (e.g. Angular, React)',
 'Data structures and algorithms for efficiently and effectively managing large data sets',
 'Serverless data warehouses (e.g. Athena) and NoSQL databases (e.g. DynamoDB)',
 'GraphQL and RESTful APIs',
 'Cloud infrastructure development using public cloud providers (e.g., AWS, Azure, GCP)',
 'Infrastructure-as-code technologies (e.g., Terraform, CloudFormation, CircleCI)',
 'Continuous integration & deployment (CI/CD), source control (e.g. GitHub), and build automation (e.g. Jenkins, GitHub Actions)',
 'Agile principles and Sprint Planning, including planning/backlog tools (e.g., JIRA)']

In [12]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links


[[{'links': 'https://github.com/qusaii21/Ganga-Prabodhanam-App'},
  {'links': 'https://github.com/qusaii21/RouteMate-Optimized-Employee-Transportation-Routing'}],
 [{'links': 'https://github.com/qusaii21/Quick-Sewa-On-Demand-House-help-Service-App'},
  {'links': 'https://github.com/qusaii21/Ganga-Prabodhanam-App'}],
 [{'links': 'https://github.com/qusaii21/minimum-cost-of-ticket'},
  {'links': 'https://github.com/qusaii21/ANPR-System-for-Apartments'}],
 [{'links': 'https://github.com/qusaii21/minimum-cost-of-ticket'},
  {'links': 'https://github.com/qusaii21/Quick-Sewa-On-Demand-House-help-Service-App'}],
 [{'links': 'https://github.com/qusaii21/Quick-Sewa-On-Demand-House-help-Service-App'},
  {'links': 'https://github.com/qusaii21/minimum-cost-of-ticket'}],
 [{'links': 'https://github.com/qusaii21/ANPR-System-for-Apartments'},
  {'links': 'https://github.com/qusaii21/Quick-Sewa-On-Demand-House-help-Service-App'}],
 [{'links': 'https://github.com/qusaii21/ANPR-System-for-Apartments'},


In [14]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Qusai Shergardwala, a third-year student at Vishwakarma Institute of Technology (VIT) Pune, pursuing a Bachelor of Technology (B.Tech) in Artificial Intelligence and Data Science.  
You are currently the Project Head at IEEE SB VIT Pune and have experience in leading technical initiatives and developing innovative solutions.

Your task is to craft a professional and compelling cold email to the client regarding the job described above. Highlight your qualifications, skills, and projects relevant to the role, demonstrating your ability to meet their requirements effectively.  
Incorporate the most relevant items from the following links to showcase your portfolio: {link_list}.  
The email should be concise, impactful, and free of any preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Application for Senior Software Engineer Position - Cloud Reporting Solution

Dear Hiring Manager,

I am excited to apply for the Senior Software Engineer position at your esteemed organization, where I can leverage my technical expertise to deliver and maintain product features for your internal cloud reporting solution. As a third-year student at Vishwakarma Institute of Technology (VIT) Pune, pursuing a Bachelor of Technology (B.Tech) in Artificial Intelligence and Data Science, I am confident that my skills and experience make me an ideal fit for this role.

As the Project Head at IEEE SB VIT Pune, I have led various technical initiatives and developed innovative solutions, honing my skills in object-oriented programming languages, including Python. My experience with JavaScript front-end web frameworks, such as React, has enabled me to create efficient and scalable UI interfaces. I am well-versed in data structures and algorithms, ensuring efficient management of large da